# Sem Eval 2013/14 Twitter sentiment task - message polarity

In [92]:
import pandas as pd
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
pd.options.display.max_rows = None
pd.options.display.notebook_repr_html = True
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 100

## Lexicon Hiu and Lu

In [27]:
#
# Get the positiv - negative word lexicon
#

import re
from sklearn.base import BaseEstimator, TransformerMixin
from nltk import word_tokenize

positive_lexicon_path = '/Users/gautamborgohain/PycharmProjects/DT_Labs/PLayground/Gautam_Borg/HuLiuLexicon/positive-words.txt'
negative_lexicon_path ='/Users/gautamborgohain/PycharmProjects/DT_Labs/PLayground/Gautam_Borg/HuLiuLexicon/negative-words.txt'
poshand = open(positive_lexicon_path)
neghand = open(negative_lexicon_path)
poslist = []
neglist = []
for line in poshand:
    poslist.append(re.sub(r'\n','',line))
for line in neghand:
    neglist.append(re.sub(r'\n','',line))
print(len(poslist),len(neglist))

poshand.close()
neghand.close()

def getPositiveWordCount(tweet):
    countPos = 0
    for word in word_tokenize(tweet):
        if len(word)>=2 and word in poslist: countPos+=1
    return countPos

def getNegativeWordCount(tweet):
    countNeg = 0
    for word in word_tokenize(tweet):
        if len(word)>=2 and word in neglist: countNeg+=1
    return countNeg

class lexiconSent(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    
    
    def transform(self, frame):
        df = pd.DataFrame(columns=['POS_LEX','NEG_LEX'])
        df['POS_LEX'] =   [getPositiveWordCount(tweet) for tweet in frame]
        df['NEG_LEX'] =   [getNegativeWordCount(tweet) for tweet in frame]
             
        return df.as_matrix()

2006 4783


## Puctuations -  Exclamations, quotes and question marks

In [177]:
class punctuations(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, frame):
        df = pd.DataFrame()
        punc_excl = []
        punc_ques = []
        punc_quot = []
        for tweet in frame:
            punc_excl.append(len(re.findall(r'!',tweet)))
            punc_ques.append(len(re.findall(r'\?',tweet)))
            punc_quot.append(len(re.findall(r'\'|"',tweet)))

        df['PUNC_EXCL'] = punc_excl
        df['PUNC_QUES'] = punc_ques
        df['PUNC_QUOT'] = punc_quot
        return df.as_matrix()

## Prepare the training and the test data

In [376]:

'''
Difference in the data


This dataset contains around 3200 missign tweets

'''

orgtrain = pd.read_csv('/Users/gautamborgohain/Desktop/SEmEval_train.tsv',sep='\t')
orgtrain.columns = ['ID','ID2','Sentiment','Tweet']
orgtest = pd.read_excel('/Users/gautamborgohain/Google Drive/CI/twitter_download20160410.xlsx')
print(len(orgtrain[orgtrain.Tweet == 'Not Available']))
print(len(orgtest[orgtest.Tweet == 'Not Available']))

2111
1137


In [3]:
semdf_train = pd.read_csv('/Users/gautamborgohain/Desktop/SEmEval_train.tsv',sep='\t')
semdf_train.columns = ['ID','ID2','Sentiment','Tweet']
semdf_train = semdf_train[semdf_train.Tweet != 'Not Available']
semdf_train.Sentiment[semdf_train.Sentiment == 'neutral'] = 0
semdf_train.Sentiment[semdf_train.Sentiment == 'positive'] = 1
semdf_train.Sentiment[semdf_train.Sentiment == 'negative'] = -1
semdf_train.Sentiment.value_counts()

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


 0    3676
 1    2814
-1    1063
Name: Sentiment, dtype: int64

In [4]:
semdf_test = pd.read_excel('/Users/gautamborgohain/Google Drive/CI/twitter_download20160410.xlsx')
# semdf_test.columns = ['ID','ID2','Sentiment','Tweet']
semdf_test = semdf_test[semdf_test.Tweet != 'Not Available']
semdf_test.Sentiment[semdf_test.Sentiment == 'neutral'] = 0
semdf_test.Sentiment[semdf_test.Sentiment == 'positive'] = 1
semdf_test.Sentiment[semdf_test.Sentiment == 'negative'] = -1
semdf_train.Sentiment =  [int(pred) for pred in semdf_train.Sentiment]
semdf_test.Sentiment.value_counts()

 0    3548
 1    2970
-1    1331
Name: Sentiment, dtype: int64

In [390]:
semdf_test.head()

,ID,USER ID,Sentiment,Tweet,Unnamed: 4
1,282031301962395008,T14111200,0,dec 21st 2012 will be know not as the end of the world but the Baby Boom! #2012shit,NaN
2,11975,SM112166,-1,Yar he quite clever but aft many guesses lor. He got ask me 2 bring but i thk darren not so will...,Yes.
3,136592,LJ112295,-1,Yeah we have Thin Lizzy here I HATE the informercials !,Not Available
4,253421252956544992,T13114433,0,"MT @LccSy #Syria, Deir Ezzor: Ali Bashar al-theeb was martyred. He was a soldier of the Free Syr...",NaN
5,220880422320603008,T14114138,-1,"@MacMiller hate my life, because i can't see you at the roskilde festival on saturday, promise m...",NaN



# filtering out the sms and the live journal dataset


In [391]:
sms = semdf_test[semdf_test['USER ID'].str.startswith('SM')]
lj = semdf_test[semdf_test['USER ID'].str.startswith('LJ')]
t = semdf_test[semdf_test['USER ID'].str.startswith('T')]

print(len(sms),len(lj),len(t))

2093 1142 4614


In [392]:
t.Sentiment.value_counts()

 1    2051
 0    1930
-1     633
Name: Sentiment, dtype: int64

Yes. I am testing my models on the test set provided in the Task and not using the development set. Shoot me.

In [393]:
X = semdf_train.Tweet
y = [int(y) for y in semdf_train.Sentiment]

Xt = t.Tweet
yt = [int(y) for y in t.Sentiment]

## Base line score

In [ ]:
LinearSVC()

In [410]:
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_recall_curve, classification_report

pipeline = Pipeline([
    ('features', FeatureUnion([
                    ('bow', CountVectorizer(stop_words='english',lowercase=True,ngram_range=(1,4)))

                ])
    ),
    ('svm', LinearSVC())    
    ])

pipeline.fit(X,y)
predictions  = pipeline.predict(Xt)
print(accuracy_score(predictions, yt)) 
print(classification_report(yt,predictions))
print(f1_score(predictions,yt,average='macro', labels = [1,-1]))

0.630905938448
             precision    recall  f1-score   support

         -1       0.66      0.16      0.26       633
          0       0.56      0.90      0.69      1930
          1       0.80      0.52      0.63      2051

avg / total       0.68      0.63      0.60      4614

0.446215783403


What are the common words in the training set?

## Added the Punctuation and Lexicon features 

In [28]:
pipeline = Pipeline([
    ('features', FeatureUnion([
                    ('bow', CountVectorizer(stop_words='english',lowercase=True,ngram_range=(1,4))),
                    ('lex',lexiconSent()),
                    ('punc',punctuations()),

                ])
    ),
    ('svm', LinearSVC())    
    ])

pipeline.fit(X,y)
predictions  = pipeline.predict(Xt)
print(accuracy_score(predictions, yt)) 

0.672060135049


## Lengths

Length of the tweet and the lengths of the largest and the smallest words in the tweet and the average length of words in the tweet

In [56]:
import math
tweet = 'sdfdf fdff g g g                                gkkl'
tweet = re.sub(r'\s+',' ',tweet)
math.log(len(tweet))

3.044522437723423

In [93]:
class lengths(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, frame):
        df = pd.DataFrame()
        df['Length'] = [len(tweet) for tweet in frame]
        df['Length_words_MAX'] = [max([len(part) for part in tweet.split(' ')]) for tweet in frame]
        df['Length_words_MIN'] = [max([len(part) for part in re.sub(r'\s+',' ',tweet).split(' ')]) for tweet in frame]
        df['Length_words_AVG'] = [np.mean([len(part) for part in tweet.split(' ')]) for tweet in frame]
        return df.as_matrix()

## regular expression to clean up the tweet


In [150]:
from sklearn.base import BaseEstimator, TransformerMixin

class regExProcesses(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, df):
        tweets = []
        for tweet in df:
#             tweet = re.sub('tomorrow|rt','',tweet)
            tweet = re.sub(r'&gt;|&amp;|&lt;','',tweet)
            tweet = re.sub(r'[\n]','',tweet)
            tweet = re.sub('[\.]+', '.', tweet)
            tweet = re.sub('@[^\s]*', '', tweet)
            tweet = re.sub('((www\.[^ ]+)|(https?://[^ ]+))', '', tweet)
            tweet = re.sub('[\s]+', ' ', tweet)
            tweets.append(tweet)
            
        return tweets
        

## MPQA Subjectivity lexicon

In [126]:
def cleantweet(tweet):
    tweet = re.sub('url|at_user|rt|\.', '', tweet)  ## removing these from the tweets
    return tweet

class subjectivityLexicon(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, df):
        lexicon = pd.read_csv(lexicon_path)
        tweet_tags = []
        count_tweet = 1
        for tweet in df:
            tweet = cleantweet(tweet)
            typeList = []
            priorpolarityList = []
            count_word = 0  
            count_tweet += 1
            for word in word_tokenize(tweet):
                result = lexicon[lexicon.word1 == word]
                if len(result) != 0:  # word is there in the lexicon
#                     if len(result) == 1:  # this case is handling the ones where the there is only one record of the word
                        typeList.append(result.iloc[0][0])
                        priorpolarityList.append(result.iloc[0][5])
#                     if len(result) > 1:  
#     #                     print('Have to tag POS, Hold On!')
#                         poslist = pos_tag(word_tokenize(tweet))#Tag the tweet
#                         postag = poslist[count_word][1]#Using the position of the word, find the POS tag
#                         if postag in ['NN', 'NNP', 'NNS',
#                                       'NNPS']:  # make the POS tags to the format used by the MPQA lexicon
#                             postag = 'noun'
#                         elif postag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
#                             postag = 'verb'
#                         elif postag in ['RB', 'RBR', 'RBS']:
#                             postag = 'adverb'
#                         elif postag in ['JJ', 'JJR', 'JJS']:
#                             postag = 'adj'
#                         second_result = result[result.pos1 == postag]#get the word-postag combination from the lexicon
#                         if len(second_result) != 0:  
#                             typeList.append(second_result.iloc[0][0])
#                             priorpolarityList.append(second_result.iloc[0][5])

                count_word += 1

            tweet_tags.append(' '.join(typeList) + ' ' + ' '.join(priorpolarityList))

        return tweet_tags


In [119]:
lexicon_path = '/Users/gautamborgohain/PycharmProjects/Twitter_target_dependent_SA/subjectivity.csv'

class subjectivityLexicon(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, df):
        lexicon = pd.read_csv(lexicon_path)
        tweet_tags = []
        count_tweet = 1
        for tweet in df:
            priorpolarityList = [lexicon[lexicon.word1 == word].priorpolarity if (len(lexicon[lexicon.word1 == word].priorpolarity) != 0) else 'None' for word in word_tokenize(tweet)]
            typeList = [lexicon[lexicon.word1 == word].type if (len(lexicon[lexicon.word1 == word].type) != 0) else 'None' for word in word_tokenize(tweet)]
            tweet_tags.append(' '.join(typeList) + ' ' + ' '.join(priorpolarityList))

        return tweet_tags


## Negations -  if its is present

In [329]:
class negations(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, frame):
        df = pd.DataFrame()
        dons = []
        nots = []
        for tweet in frame:
            dons.append(len(re.findall(' don | dont | don\'t | never | no | not | neither | nor | none ',tweet)))
            nots.append(len(re.findall(' not ',tweet))) 
        
        df = pd.DataFrame({'dons' : dons,'nots': nots})
        return df.as_matrix()

## Emoticons

In [133]:
class emoticons(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, frame):
        df = pd.DataFrame()
        posEmot,negEmot,posEmo,negEmo,neutralEmo = [],[],[],[],[]
        for tweet in frame:
            posEmot.append(len(re.findall(r':-D|:D|:-\)|:\)|;-\)|;\)',tweet)))
            negEmot.append(len(re.findall(r':-\(|:\(',tweet)))
            
        df['EMOT_POS'] = posEmot
        df['EMOT_NEG'] = negEmot
        
        return df.as_matrix()

## Word elongations - 3 letters consecutive

- The no of elongations in the tweet
- The average of the elongation length in the tweet

In [173]:
class elongation(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, frame):
        df = pd.DataFrame()
        no_elong = []
        avg_elong = []
        reg = re.compile('([a-zA-Z])\\1{3,}')
        for tweet in frame:
            elongs = []
            no_elong.append(len(reg.findall(tweet)))
            for match in reg.finditer(tweet):
                                     elongs.append((match.end() - match.start()))
                                     
            avg_elong.append(np.mean(elongs))                         
        df['No_Elong'] = no_elong
        df['Avg_Elong'] = avg_elong
        df.fillna(value=0,inplace=True)
        return df.as_matrix()

In [164]:
tweet = 'I am sooooo going to killllllll yoiu'
for match in re.finditer("([a-zA-Z])\\1{2,}",tweet):
    print(match.end() - match.start())

5
8


In [195]:
 def elongs(frame):
        df = pd.DataFrame()
        no_elong = []
        avg_elong = []
        reg = re.compile('([a-zA-Z])\\1{3,}')
        for tweet in frame:
            elongs = []
            no_elong.append(len(reg.findall(tweet)))
            for match in reg.finditer(tweet):
                                     elongs.append((match.end() - match.start()))
                                     
            avg_elong.append(np.mean(elongs))                         
        df['No_Elong'] = no_elong
        df['Avg_Elong'] = avg_elong
        df.fillna(value=0,inplace=True)
        return df
    
df = elongs(X[0:100])
df.describe()

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


,No_Elong,Avg_Elong
count,100.000000,100.000000
mean,0.060000,0.226667
std,0.342893,1.189064
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,3.000000,9.000000


## Hastags in the tweet


In [215]:
class hash_tag(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, frame):
        return [' '.join(re.findall(r'#[^\s]+',tweet)) for tweet in frame]

In [204]:
def hash_vect(frame):
        df = pd.DataFrame()
        hash_pos_all = []
        hash_neg_all = []
        hash_all = []
        for tweet in frame:
            hash_pos = []
            hash_neg = []
            hash_inTweet = []
            hashs = re.findall(r'#[^\s]+',tweet)
            for hashe in hashs:
                if hashe in poslist:
                    hash_pos.append(hashe)
                if hashe in neglist:
                    hash_neg.append(hashe)
                hash_inTweet.append(hashe)
            hash_pos_all.append(' '.join(hash_pos))
            hash_neg_all.append(' '.join(hash_neg))  
            hash_all.append(' '.join(hash_inTweet))        
        df['HASH_POS'] = hash_pos_all
        df['HASH_NEG'] = hash_neg_all
        df['HASH_ALL'] = hash_all        
        return df
df = hash_vect(X)
df.describe()

,HASH_POS,HASH_NEG,HASH_ALL
count,7553,7553,7553
unique,1,1,1397
top,,,
freq,7553,7553,6022


The lexicon does not have the sentiment if any of the hash tags in the training set, so just taking the hash tags as is

In [210]:
'stressfull' in neglist

False

In [315]:
from sklearn.base import BaseEstimator, TransformerMixin

class urls_flag(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, frame):
        df = pd.DataFrame()
        urls = []
        for tweet in frame:
            urls.append(len(re.findall('((www\.[^ ]+)|(https?://[^ ]+))', tweet)))
        df['URLS'] = urls
        return df

In [293]:
tweet = 'I think I may have a heart attack for Jason Wus new collection. So Charlotte Rampling in the Night  http://t.co/2KRW4kTn'
len(re.findall('((www\.[^ ]+)|(https?://[^ ]+))', tweet))

1

In [310]:
def urls(frame):
    df = pd.DataFrame()
    urls = []
    for tweet in frame:
        urls.append(len(re.findall('((www\.[^ ]+)|(https?://[^ ]+))', tweet)))
    df['URLS'] = urls
    return df
print(len(X))
df = urls(X)


7553


In [318]:
from sklearn.base import BaseEstimator, TransformerMixin

class remove_url(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, frame):
        return [re.sub('((www\.[^ ]+)|(https?://[^ ]+))','',tweet) for tweet in frame]

In [339]:
from nltk import pos_tag
from nltk import word_tokenize

class posTagTweets(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, df):
        tagsoftweet = []
        for tweet in df:
            postaggedtweet = pos_tag(word_tokenize(tweet))  # this one is pos atgged..list inside list : token[1] for tag
            tags = []
            for token in postaggedtweet:
                tags.append(token[1])
            tagsoftweet.append(' '.join(tags))
#             print(' '.join(tags))
            
#         df = getFeatureDF(tagsoftweet)
        return tagsoftweet

In [351]:
class otherFeats(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, frame):
        df = pd.DataFrame()
        no_of_sents = []
        no_of_newlines = []
        no_of_rsdots = []
        no_of_mentions = []
        no_of_rspaces = []
        for tweet in frame:
            no_of_sents.append(len(tweet.split('.')))
            no_of_newlines.append(len(re.findall(r'[\n]',tweet)))
            no_of_rsdots.append(len(re.findall('[\.]+', tweet)))
            no_of_mentions.append(len(re.findall('@[^\s]*', tweet)))
            no_of_rspaces.append(len(re.findall('[\s]+', tweet)))
            
        df['SENTS'] = no_of_sents
        df['NEWLINES'] = no_of_newlines
        df['DOTS'] = no_of_rsdots
        df['MENTIONS'] = no_of_mentions
        df['SPACES'] = no_of_rspaces
        return df.as_matrix()

In [367]:
df=  pd.DataFrame({'yt': yt,'preds' : predictions})
pd.crosstab(df.yt,df.preds)


preds,-1,0,1
yt,,,
-1,566,626,139
0,157,3097,294
1,181,1091,1698


In [409]:
from nltk.tokenize import TreebankWordTokenizer
pipeline = Pipeline([
#     ('regex', regExProcesses()),
    ('features', FeatureUnion([
                    ('bow', Pipeline([
                                ('clean',remove_url()),
                                ('vect',CountVectorizer(stop_words='english',lowercase=True,ngram_range=(1,4),tokenizer=TreebankWordTokenizer().tokenize))
                                ])),
                    ('tfbow', Pipeline([
                                    ('clean',remove_url()),
                                    ('tempvect', CountVectorizer()),
                                    ('tfvect',TfidfTransformer())
                                ])),                    
                    ('cow', Pipeline([
                                    ('clean',remove_url()),
                                    ('cvect',CountVectorizer(analyzer = 'char_wb',lowercase=True,ngram_range=(2,2)))
                                ])),
                    ('lex',lexiconSent()),
                    ('punc',punctuations()),
                    ('len',lengths()),
                    ('negs',negations()),
                    ('emots',emoticons()),
                    ('elongs',elongation()),
                    ('hashs',Pipeline([
                                ('hashs',hash_tag()),
                                ('hash_vect',CountVectorizer())
                            ])),
                    ('urls',urls_flag()),
                    ('negativs', negatives()),
                    ('counts',otherFeats())

                ])
    ),
    ('svm', LinearSVC())    
    ])

pipeline.fit(X,y)
predictions  = pipeline.predict(Xt)
print(accuracy_score(predictions, yt)) 
print(f1_score(predictions,yt,average='macro'))
print(classification_report(yt,predictions))
print(f1_score(predictions,yt,average='macro', labels = [1,-1]))

0.668617251842
0.617283050158
             precision    recall  f1-score   support

         -1       0.61      0.38      0.47       633
          0       0.61      0.83      0.70      1930
          1       0.78      0.61      0.68      2051

avg / total       0.68      0.67      0.66      4614

0.573591828758


/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


In [408]:
from nltk.tokenize import TreebankWordTokenizer
pipeline = Pipeline([
#     ('regex', regExProcesses()),
    ('features', FeatureUnion([
                    ('bow', Pipeline([
                                ('clean',remove_url()),
                                ('vect',CountVectorizer(stop_words='english',lowercase=True,ngram_range=(1,4),tokenizer=TreebankWordTokenizer().tokenize))
                                ])),
#                     ('tfbow', Pipeline([
#                                     ('clean',remove_url()),
#                                     ('tempvect', CountVectorizer()),
#                                     ('tfvect',TfidfTransformer())
#                                 ])),                    
#                     ('cow', Pipeline([
#                                     ('clean',remove_url()),
#                                     ('cvect',CountVectorizer(analyzer = 'char_wb',lowercase=True,ngram_range=(2,2)))
#                                 ])),
                    ('lex',lexiconSent()),
                    ('punc',punctuations()),
                    ('len',lengths()),
#                     ('negs',negations()),
                    ('emots',emoticons()),
#                     ('elongs',elongation()),
                    ('hashs',Pipeline([
                                ('hashs',hash_tag()),
                                ('hash_vect',CountVectorizer())
                            ])),
                    ('urls',urls_flag()),
#                     ('negativs', negatives()),
#                     ('counts',otherFeats())

                ])
    ),
    ('svm', LinearSVC())    
    ])

pipeline.fit(X,y)
predictions  = pipeline.predict(Xt)
print(accuracy_score(predictions, yt)) 
print(f1_score(predictions,yt,average='macro'))
print(classification_report(yt,predictions))
print(f1_score(predictions,yt,average='macro', labels = [1,-1]))

0.652795838752
0.583511220486
             precision    recall  f1-score   support

         -1       0.70      0.28      0.40       633
          0       0.57      0.91      0.70      1930
          1       0.83      0.52      0.64      2051

avg / total       0.71      0.65      0.64      4614

0.523266830729
